## Lux is now supporting postgreSQL database (April 2021)

In the tutorial, we expand on current capabilities of lux to support postgresSQL--one of the popular relational databases used by communities of data professionals. We will show how users can connect their database tables and views to a LuxSQLTable to leverage the rich recommendation capabilities provided by Lux without having to pull all of the data into a dataframe. You are free to use this tutorial on Binder or set up the test database on your own environment and play with the notebook locally. By the end of the tutorial, you will be able to:

- Create a new database and database table within a postgreSQL server
- Connect to the database via Lux
- Create a LuxSQLTable object for lux to provide rich recommendations
- Get a refresher of lux' functionality with the employee demo

Note that Lux' SQL features are still a work in progress, only PostgreSQL is currently supported and tested.

## A. Setup in 3 steps:
- 1) Create PostgreSQL server connection and link that to Lux
- 2) Create LuxSQLTable object
- 3) Check installed packages

## 1) Create PostgreSQL and connect to the database table

In this environment, we have set up a PostgreSQL database and populated it the ```employee``` table used in this demo. If you don't have PostgreSQL server setup, you can get started [here](https://www.postgresql.org).


For those with PostgreSQL, we setup the PostgreSQL connection and configure the connection.

In [ ]:
import lux
import pandas as pd
from sqlalchemy import create_engine
import psycopg2

connection = psycopg2.connect("host=localhost dbname=testdb user=testuser password=testpass")
lux.config.set_SQL_connection(connection)

## 2) Create LuxSQLtable object

We create a LuxSQLtable named ```tbl``` then specify the table we are using in the demo to be ```employee```.

In [ ]:
tbl = lux.LuxSQLTable()
tbl.set_SQL_table("employee")

If you are running the jupyter notebook locally, you can see the changes here:
- https://mybinder.org/v2/gh/lux-org/lux-binder-sql/HEAD
You can find the link to the script and it's also pasted below: https://github.com/thyneb19/lux-binder-sql/blob/master/init_db/postgres/init_db.sh

In [ ]:
#!/bin/bash
set -eux

THISDIR="$( cd "$( dirname "${BASH_SOURCE[0]}" )" && pwd )"

#Demo PostgreSQL Database initialisation
psql postgres -c "CREATE USER testuser PASSWORD 'testpass'"
#The -O flag below sets the user: createdb -O DBUSER DBNAME
createdb -O testuser testdb

psql -d testdb -U testuser -f $THISDIR/seed_db.sql

## 3) Check installed packages
Currently, lux's recommendation capability is expanded to support postgreSQL, so check you have all the installation of necessary packages below.

In [ ]:
import pandas as pd
import lux
from sqlalchemy import create_engine
import psycopg2
from lux.vis.Vis import Vis
from lux.vis.VisList import VisList

Woot! Setup is now complete. Let's explore these datasets using lux. The following is an adaptation from the [employee demo](https://github.com/lux-org/lux-binder/blob/master/demo/employee_demo.ipynb).

In this demo, we will be analyzing the [IBM HR Employee Attrition dataset](https://www.kaggle.com/pavansubhasht/ibm-hr-analytics-attrition-dataset) to understand how employee characteristics might influence attrition. 

Lux is designed to be tightly integrated with Pandas and can be used as-is, without modifying your existing Pandas code. 

In [ ]:
# Collecting basic usage statistics for Lux (For more information, see: https://tinyurl.com/logging-consent)
lux.logger = True # Remove this line if you do not want your interactions recorded

<div>
    <img align="left" src="https://github.com/lux-org/lux-resources/blob/master/icon/table.png?raw=True" width="30">
    <h1 style="padding-left: 40px;">Visualizations of database tables</h1>
</div>

To get an overview of the database table, simply print out the LuxSQLTable `tbl`. By clicking on the Toggle button, you can now explore the data visually through Lux. You should see three tabs of visualizations recommended to you. 

In [ ]:
tbl

The visualizations are displayed in different tabs as [actions](https://lux-api.readthedocs.io/en/latest/source/getting_started/overview.html#visualizing-dataframes-with-recommendations).
By inspecting the Correlation action, we see several salient patterns with a sharp triangular pattern. This checks out with our intuition that it is impossible for total working years to exceed the employee's age, and for any employee to stay longer at a company than their total working years.  

<div>
    <img align="left" src="https://github.com/lux-org/lux-resources/blob/master/icon/steering-wheel.png?raw=True" width="30">
    <h1 style="padding-left: 40px;">Steering analysis with intent</h1>
</div>

Let's say that we want to investigate factors that influence employee attrition. Beyond these basic recommendations, you can further specify your analysis *intent*, i.e., the data attributes and values that you are interested in visualizing. 

In [ ]:
tbl.intent=["Attrition"]

Upon printing the dataframe again, Lux leverages the analysis intent to steer the recommendations towards what the user might be interested in.

In [ ]:
tbl

On the left, we see that the visualization based on the specified intent shows that around 15% of employees leaves the company. On the right, in the Enhance action, we learn that employees that leave typically spent less time in their working life and at the Company than their counterparts by about four years. 

ℹ️ Check out [this tutorial](https://lux-api.readthedocs.io/en/latest/source/guide/intent.html#) to learn more about how to specify intent in Lux.

<div>
    <img align="left" src="https://github.com/lux-org/lux-resources/blob/master/icon/search.png?raw=True" width="30">
    <h1 style="padding-left: 40px;">Quick inspection of 1-D Series</h1>
</div>

Based on this insight, we want to derive a new column that captures what percentage of the employee's working year have they spend at this company. We quickly divide the two columns in order to inspect the Series visualization. 

In [ ]:
tbl["YearsAtCompany"]/tbl["TotalWorkingYears"]

We print out the result of dividing the two columns. The Series visualization enables us to quickly verify that the data range lies between 0 and 1. Somewhat surprisingly, we also find that a large group of employees who have spent almost all of their working life at this company. 

We want to learn more, so we create a new column `%WorkingYearsAtCompany` to capture this metric.

In [ ]:
tbl["%WorkingYearsAtCompany"]=tbl["YearsAtCompany"]/tbl["TotalWorkingYears"]

<div>
    <img align="left" src="https://github.com/lux-org/lux-resources/blob/master/icon/flask.png?raw=True" width="30">
    <h1 style="padding-left: 40px;">Quick-and-dirty experimentation with visualizations</h1>
</div>


Lux is built on the principle that users should always be able to visualize and explore anything they specify, without having to think about how the visualization should look like.

Continuing our analysis, we are interested in seeing if there are any differences in percentage of working years at the Company for young employees compared to older employees. To investigate this hypothesis, we generate a Vis object showing the relationship between `Age` and `%WorkingYearsAtCompany`. 

In [ ]:
Vis(["%WorkingYearsAtCompany","Age"],tbl)

The visualization does not show a super clear trend. To look into this more, we can binarize the `Age` variable based on whether the employee is above or below the average age across employees. 

In [ ]:
tbl["Age"]

In [ ]:
tbl["IsOld"]=tbl["Age"]>tbl["Age"].mean()

In [ ]:
tbl["IsOld"]

Now, we again look at whether young or old people tend to stay in the company for longer. The visualization generated from Lux shows that older employees actually have a shorter percentage of working year at the company. 

In [ ]:
Vis(["%WorkingYearsAtCompany","IsOld"],tbl)

We are also interested in whether the salary of old and young employees differs significantly. We take a look at the columns in the dataframe and see that there is a group of columns that contains the word `Rate`.

In [ ]:
tbl.columns

In [ ]:
compensation = list(filter(lambda col: 'Rate' in col, tbl.columns))
compensation

We then pose these compensation-related attributes against the `IsOld` variable and find that the differences in compensation is not significant across old v.s. young employees. 

In [ ]:
VisList([compensation,"IsOld"],tbl)

The programmatic generation of Vis and Vislist provides a quick and dirty way to ask specific questions about the dataframe.

ℹ️ Check out [this tutorial](https://lux-api.readthedocs.io/en/latest/source/guide/vis.html) to learn more about how to create Vis and VisList in Lux.

<div>
    <img align="left" style="margin-top: -10px;" src="https://github.com/lux-org/lux-resources/blob/master/icon/present.png?raw=True" width="50">
    <h1 style="padding-left: 55px;">Exporting visualization insight to edit and share</h1>
</div>

By skimming through the recommended visualizations again, we find that distance from home to work is a important factor in employee attrition.

In [ ]:
tbl

We can click on the bar chart visualization of `DistanceFromHome` v.s. `Attrition` and export it as code.

In [ ]:
vis = tbl.exported[0]
print(vis.to_code(language="altair"))

Then we can make minor stylistic changes before we can export this visualization to a slidedeck to share with colleagues.

In [ ]:
import altair as alt
visData = pd.DataFrame({'Attrition': {0: 'No', 1: 'Yes'}, 
                        'DistanceFromHome': {0: 8.915652879156529, 1: 10.632911392405063}})

chart = alt.Chart(visData,title="Important Factor for Employee Attrition!!").mark_bar().encode(
    y = alt.Y('Attrition', type= 'nominal', axis=alt.Axis(labelOverlap=True, title='Attrition')),
    x = alt.X('DistanceFromHome', type= 'quantitative', axis=alt.Axis(title='Average Distance from Home')),
)
chart = chart.configure_mark(tooltip=alt.TooltipContent('encoding'))
chart = chart.configure_title(fontWeight=500,fontSize=13,font='Helvetica Neue')
chart = chart.configure_axis(titleFontWeight=500,titleFontSize=11,titleFont='Helvetica Neue',
            labelFontWeight=400,labelFontSize=8,labelFont='Helvetica Neue',labelColor='#505050')
chart = chart.configure_legend(titleFontWeight=500,titleFontSize=10,titleFont='Helvetica Neue',
            labelFontWeight=400,labelFontSize=8,labelFont='Helvetica Neue')
chart = chart.properties(width=250,height=70)

chart

ℹ️ Check out [this tutorial](https://lux-api.readthedocs.io/en/latest/source/guide/export.html) to learn more about exporting visualizations in Lux.


# Try out Lux! 

To get started, Lux can be installed through [PyPI](https://pypi.org/project/lux-api/). 

```bash
pip install lux-api
``` 


To use Lux in [Jupyter notebook](https://github.com/jupyter/notebook) or [VSCode](https://code.visualstudio.com/docs/python/jupyter-support), activate the notebook extension:

```bash
jupyter nbextension install --py luxwidget
jupyter nbextension enable --py luxwidget
```

To use Lux in [Jupyter Lab](https://github.com/jupyterlab/jupyterlab), activate the lab extension:

```bash
jupyter labextension install @jupyter-widgets/jupyterlab-manager
jupyter labextension install luxwidget
```

If you encounter issues with the installation, please refer to [this page](https://lux-api.readthedocs.io/en/latest/source/guide/FAQ.html#troubleshooting-tips) to troubleshoot the installation.

### More information: 

- Follow us on [Twitter](https://twitter.com/lux_api) for discussion and updates.
- Sign up for the early-user [mailing list](https://forms.gle/XKv3ejrshkCi3FJE6) to stay tuned for upcoming releases, updates, or user studies. 
- Visit [ReadTheDoc](https://lux-api.readthedocs.io/en/latest/) for more detailed documentation.
- Clone [lux-binder](https://github.com/lux-org/lux-binder) to try out these [hands-on exercises](https://github.com/lux-org/lux-binder/tree/master/exercise) or [tutorial series](https://github.com/lux-org/lux-binder/tree/master/tutorial) on how to use Lux.
- Report any bugs, issues, or requests through [Github Issues](https://github.com/lux-org/lux/issues). 

<div style="color:#bfbfbf">Icons made by <a href="https://www.freepik.com" title="Freepik" style="color:#b1bbe6">Freepik</a> from <a href="https://www.flaticon.com/" title="Flaticon" style="color:#b1bbe6">www.flaticon.com</a></div>